In [2]:
import readpdf_data
import pandas as pd
import re
rawdata = readpdf_data.returndata()

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [3]:
rawdata.rename(columns={"Account Name:": "Account_Name", "Date & Time": "Date_Time","Activity Details":"Activity_Details","Amount ($ / Meals)":"Amount"},inplace=True)
rawdata['Date_Time'] = pd.to_datetime(rawdata['Date_Time'])
rawdata = rawdata[rawdata['Activity_Details'] != 'Meal Plan Office']
pd.reset_option('display.max_rows')
(rawdata)





C:\Users\ianmm\AppData\Local\Temp\ipykernel_8832\3926202341.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  rawdata['Date_Time'] = pd.to_datetime(rawdata['Date_Time'])


,Account_Name,Date_Time,Activity_Details,Amount,Balance
0,E Unlimited 250,2023-11-28 17:22:00,WorcDC3,-1,361
1,E Unlimited 250,2023-11-28 14:55:00,WorcDC3,-1,362
2,E Unlimited 250,2023-11-28 10:23:00,FrankDC1,-1,363
3,E Unlimited 250,2023-11-28 09:50:00,FrankDC2,-1,364
4,E Unlimited 250,2023-11-27 21:30:00,WorcDC3,-1,365
...,...,...,...,...,...
436,E Unlimited 250,2023-09-02 11:27:00,BerkDC3,-1,595
437,E Unlimited 250,2023-09-01 19:55:00,HampDC2,-1,596
438,E Unlimited 250,2023-09-01 12:06:00,WorcDC4,-1,597
439,E Unlimited 250,2023-08-31 19:52:00,BerkDC3,-1,598


In [21]:
laundry = rawdata[rawdata['Activity_Details'].str.contains("WASHER|DRYER")]
laundrywash = rawdata[rawdata['Activity_Details'].str.contains("WASHER")]
laundrydry = rawdata[rawdata['Activity_Details'].str.contains("DRYER")]
display(laundrydry)
most_common_dryer = laundrydry['Activity_Details'].value_counts().idxmax()
display(laundrywash)
most_common_washer = laundrywash['Activity_Details'].value_counts().idxmax()
print(most_common_dryer)
print(most_common_washer)
most_common_day = laundry['Date_Time'].dt.day_name().value_counts().idxmax()
print(most_common_day)

,Account_Name,Date_Time,Activity_Details,Amount,Balance
69,Student Debit Plan,2023-11-08 16:06:00,OAK DRYER 02,-$1.40,$5.60
121,Student Debit Plan,2023-10-29 16:04:00,OAK DRYER 08,-$1.40,$9.80
162,Student Debit Plan,2023-10-18 15:25:00,OAK DRYER 07,-$1.40,$4.00
203,Student Debit Plan,2023-10-09 17:11:00,OAK DRYER 02,-$1.40,$8.20
272,Student Debit Plan,2023-09-27 12:06:00,OAK DRYER 10,-$1.40,$12.40
273,Student Debit Plan,2023-09-27 12:06:00,OAK DRYER 09,-$1.40,$11.00
368,Student Debit Plan,2023-09-13 17:20:00,OAK DRYER 09,-$1.40,$1.60
409,Student Debit Plan,2023-09-06 12:47:00,OAK DRYER 10,-$1.40,$5.80


,Account_Name,Date_Time,Activity_Details,Amount,Balance
70,Student Debit Plan,2023-11-08 14:45:00,OAK WASHER 07,-$1.40,$8.40
71,Student Debit Plan,2023-11-08 14:45:00,OAK WASHER 02,-$1.40,$7.00
124,Student Debit Plan,2023-10-29 15:26:00,OAK WASHER 06,-$1.40,$2.60
125,Student Debit Plan,2023-10-29 15:26:00,OAK WASHER 07,-$1.40,$1.20
163,Student Debit Plan,2023-10-18 14:48:00,OAK WASHER 05,-$1.40,$5.40
164,Student Debit Plan,2023-10-18 14:47:00,OAK WASHER 07,-$1.40,$6.80
204,Student Debit Plan,2023-10-09 16:34:00,OAK WASHER 07,-$1.40,$9.60
274,Student Debit Plan,2023-09-27 11:28:00,OAK WASHER 08,-$1.40,$13.80
275,Student Debit Plan,2023-09-27 11:27:00,OAK WASHER 07,-$1.40,$15.20
369,Student Debit Plan,2023-09-13 16:43:00,OAK WASHER 07,-$1.40,$3.00


OAK DRYER 02
OAK WASHER 07
Wednesday


In [5]:

alldininghalls = ["HampDC1","HampDC2","HampDC3","HampDC4","BerkDC1","BerkDC2","BerkDC3","BerkDC4","FrankDC1","FrankDC2","FrankDC3","FrankDC4","WorcDC1","WorcDC2","WorcDC3","WorcDC4"]
if(rawdata['Account_Name'] == "E Unlimited 250").any() | (rawdata['Account_Name'] == "M Unlimited DC").any():
    mealswipes = rawdata[rawdata['Account_Name'].isin(["E Unlimited 250","M Unlimited DC"])]
    #,"C Basic Plan","UMass Dining Dollars","R YCMP Commuter Go"
else:
    mealswipes = rawdata[rawdata['Activity_Details'].isin(alldininghalls)]

mealswipes = mealswipes[mealswipes['Activity_Details'] != "Meal Plan Office"]

display(mealswipes)

mealswipes.loc[:, 'Balance'] = mealswipes['Balance'].str.replace('$', '')
# dining_dollars.loc[:, 'Amount'] = dining_dollars['Amount'].str.replace('-', '')
mealswipes['Balance'] = pd.to_numeric(mealswipes['Balance'])
mealswipes = mealswipes.reset_index(drop=True)

#mealswipes.drop_duplicates(subset='Balance', keep='first', inplace=True)

,Account_Name,Date_Time,Activity_Details,Amount,Balance
0,E Unlimited 250,2023-11-28 17:22:00,WorcDC3,-1,361
1,E Unlimited 250,2023-11-28 14:55:00,WorcDC3,-1,362
2,E Unlimited 250,2023-11-28 10:23:00,FrankDC1,-1,363
3,E Unlimited 250,2023-11-28 09:50:00,FrankDC2,-1,364
4,E Unlimited 250,2023-11-27 21:30:00,WorcDC3,-1,365
...,...,...,...,...,...
436,E Unlimited 250,2023-09-02 11:27:00,BerkDC3,-1,595
437,E Unlimited 250,2023-09-01 19:55:00,HampDC2,-1,596
438,E Unlimited 250,2023-09-01 12:06:00,WorcDC4,-1,597
439,E Unlimited 250,2023-08-31 19:52:00,BerkDC3,-1,598


In [6]:
hamp = mealswipes[mealswipes['Activity_Details'].isin(["HampDC1","HampDC2","HampDC3","HampDC4"])]
hamp.reset_index(drop=True,inplace=True)
berk = mealswipes[mealswipes['Activity_Details'].isin(["BerkDC1","BerkDC2","BerkDC3","BerkDC4"])]
berk.reset_index(drop=True,inplace=True)
frank = mealswipes[mealswipes['Activity_Details'].isin(["FrankDC1","FrankDC2","FrankDC3","FrankDC4"])]
frank.reset_index(drop=True,inplace=True)
woo = mealswipes[mealswipes['Activity_Details'].isin(["WorcDC1","WorcDC2","WorcDC3","WorcDC4"])]
woo.reset_index(drop=True,inplace=True)

misc_mealswipes = mealswipes[~mealswipes['Activity_Details'].isin(alldininghalls)]
misc_mealswipes.reset_index(drop=True,inplace=True)

In [7]:
display(hamp,berk,frank,woo,misc_mealswipes)

,Account_Name,Date_Time,Activity_Details,Amount,Balance
0,E Unlimited 250,2023-11-27 15:06:00,HampDC4,-1,366
1,E Unlimited 250,2023-11-26 19:24:00,HampDC2,-1,367
2,E Unlimited 250,2023-11-20 07:48:00,HampDC2,-1,371
3,E Unlimited 250,2023-11-19 15:00:00,HampDC2,-1,373
4,E Unlimited 250,2023-11-17 20:21:00,HampDC2,-1,377
...,...,...,...,...,...
58,E Unlimited 250,2023-09-05 20:03:00,HampDC4,-1,587
59,E Unlimited 250,2023-09-05 14:03:00,HampDC2,-1,588
60,E Unlimited 250,2023-09-03 18:07:00,HampDC2,-1,593
61,E Unlimited 250,2023-09-02 17:40:00,HampDC2,-1,594


,Account_Name,Date_Time,Activity_Details,Amount,Balance
0,E Unlimited 250,2023-11-20 12:27:00,BerkDC1,-1,369
1,E Unlimited 250,2023-11-20 12:27:00,BerkDC1,-1,370
2,E Unlimited 250,2023-11-19 21:57:00,BerkDC3,-1,372
3,E Unlimited 250,2023-11-17 23:44:00,BerkDC3,-1,376
4,E Unlimited 250,2023-11-04 23:22:00,BerkDC3,-1,411
5,E Unlimited 250,2023-11-04 16:57:00,BerkDC3,-1,412
6,E Unlimited 250,2023-11-02 22:26:00,BerkDC3,-1,416
7,E Unlimited 250,2023-10-29 21:37:00,BerkDC3,-1,426
8,E Unlimited 250,2023-10-28 13:45:00,BerkDC3,-1,429
9,E Unlimited 250,2023-10-28 12:59:00,BerkDC3,-1,430


,Account_Name,Date_Time,Activity_Details,Amount,Balance
0,E Unlimited 250,2023-11-28 10:23:00,FrankDC1,-1,363
1,E Unlimited 250,2023-11-28 09:50:00,FrankDC2,-1,364
2,E Unlimited 250,2023-11-16 10:27:00,FrankDC1,-1,381
3,E Unlimited 250,2023-11-16 09:52:00,FrankDC2,-1,382
4,E Unlimited 250,2023-11-09 10:13:00,FrankDC1,-1,398
5,E Unlimited 250,2023-11-09 10:13:00,FrankDC1,-1,399
6,E Unlimited 250,2023-11-07 10:14:00,FrankDC1,-1,403
7,E Unlimited 250,2023-11-07 10:14:00,FrankDC1,-1,404
8,E Unlimited 250,2023-11-07 10:05:00,FrankDC1,-1,405
9,E Unlimited 250,2023-10-31 17:39:00,FrankDC3,-1,421


,Account_Name,Date_Time,Activity_Details,Amount,Balance
0,E Unlimited 250,2023-11-28 17:22:00,WorcDC3,-1,361
1,E Unlimited 250,2023-11-28 14:55:00,WorcDC3,-1,362
2,E Unlimited 250,2023-11-27 21:30:00,WorcDC3,-1,365
3,E Unlimited 250,2023-11-26 14:38:00,WorcDC3,-1,368
4,E Unlimited 250,2023-11-18 21:03:00,WorcDC3,-1,374
...,...,...,...,...,...
86,E Unlimited 250,2023-09-08 13:36:00,WorcDC3,-1,579
87,E Unlimited 250,2023-09-07 17:47:00,WorcDC4,-1,581
88,E Unlimited 250,2023-09-06 20:10:00,WorcDC3,-1,584
89,E Unlimited 250,2023-09-06 15:00:00,WorcDC3,-1,585


,Account_Name,Date_Time,Activity_Details,Amount,Balance
0,E Unlimited 250,2023-11-14 09:57:00,Roots Cafe 1,-1,387
1,E Unlimited 250,2023-11-02 08:18:00,Roots Cafe 1,-1,418
2,E Unlimited 250,2023-10-31 08:20:00,Roots Cafe 1,-1,423
3,E Unlimited 250,2023-10-27 08:48:00,Roots Cafe 1,-1,432
4,E Unlimited 250,2023-10-24 08:27:00,Roots Cafe 1,-1,440
5,E Unlimited 250,2023-10-18 09:52:00,Roots Cafe 1,-1,452
6,E Unlimited 250,2023-10-17 08:25:00,Roots Cafe 1,-1,455
7,E Unlimited 250,2023-10-12 08:18:00,Roots Cafe 1,-1,463
8,E Unlimited 250,2023-10-11 10:05:00,Roots Cafe 1,-1,466
9,E Unlimited 250,2023-10-06 08:46:00,Roots Cafe 1,-1,478


In [8]:
hamp_breakfast = len(hamp[hamp['Date_Time'].dt.time < pd.to_datetime('11:00').time()])
hamp_lunch = len(hamp[(hamp['Date_Time'].dt.time >= pd.to_datetime('11:00').time()) & (hamp['Date_Time'].dt.time < pd.to_datetime('16:30').time())])
hamp_dinner = len(hamp[hamp['Date_Time'].dt.time >= pd.to_datetime('16:30').time()])
proccessed_meal_swipe_data = pd.DataFrame({"Hamp":[hamp_breakfast,hamp_lunch,hamp_dinner,'NA',len(hamp)]})


berk_breakfast = len(berk[berk['Date_Time'].dt.time < pd.to_datetime('11:00').time()])
berk_lunch = len(berk[(berk['Date_Time'].dt.time >= pd.to_datetime('11:00').time()) & (berk['Date_Time'].dt.time < pd.to_datetime('16:30').time())])
berk_dinner = len(berk[(berk['Date_Time'].dt.time >= pd.to_datetime('16:30').time()) & (berk['Date_Time'].dt.time < pd.to_datetime('21:00').time())])
berk_latenight = len(berk[(berk['Date_Time'].dt.time >= pd.to_datetime('21:00').time())])
proccessed_meal_swipe_data['Berk'] = [berk_breakfast,berk_lunch,berk_dinner,berk_latenight,len(berk)]

woo_breakfast = len(woo[woo['Date_Time'].dt.time < pd.to_datetime('11:00').time()])
woo_lunch = len(woo[(woo['Date_Time'].dt.time >= pd.to_datetime('11:00').time()) & (woo['Date_Time'].dt.time < pd.to_datetime('16:30').time())])
woo_dinner = len(woo[(woo['Date_Time'].dt.time >= pd.to_datetime('16:30').time()) & (woo['Date_Time'].dt.time < pd.to_datetime('21:00').time())])
woo_latenight = len(woo[(woo['Date_Time'].dt.time >= pd.to_datetime('21:00').time())])
proccessed_meal_swipe_data['Woo'] = [woo_breakfast,woo_lunch,woo_dinner,woo_latenight,len(woo)]

frank_breakfast = len(frank[frank['Date_Time'].dt.time < pd.to_datetime('11:00').time()])
frank_lunch = len(frank[(frank['Date_Time'].dt.time >= pd.to_datetime('11:00').time()) & (frank['Date_Time'].dt.time < pd.to_datetime('16:30').time())])
frank_dinner = len(frank[(frank['Date_Time'].dt.time >= pd.to_datetime('16:30').time())])
proccessed_meal_swipe_data['Frank'] = [frank_breakfast,frank_lunch,frank_dinner,'NA',len(frank)]

mics_columns = misc_mealswipes['Activity_Details'].unique()
for place in mics_columns:
    pattern = r'[0-9]'
    micsdf = misc_mealswipes[misc_mealswipes['Activity_Details'].isin([place])]
    num_breakfast = len(micsdf[micsdf['Date_Time'].dt.time < pd.to_datetime('11:00').time()])
    num_lunch = len(micsdf[(micsdf['Date_Time'].dt.time >= pd.to_datetime('11:00').time()) & (micsdf['Date_Time'].dt.time < pd.to_datetime('16:30').time())])
    num_dinner = len(micsdf[micsdf['Date_Time'].dt.time >= pd.to_datetime('16:30').time()])

    place = re.sub(pattern, '', place)
    proccessed_meal_swipe_data[place] = [num_breakfast,num_lunch,num_dinner,'NA',len(micsdf)]

proccessed_meal_swipe_data.rename(index={0: "Breakfast", 1: "Lunch", 2: "Dinner",3:"Late_Night",4:"Total"},inplace=True)
total_meal_swpes = proccessed_meal_swipe_data.iloc[4].sum(axis=0)
print(f"youve used {total_meal_swpes} meal swipes")
display(proccessed_meal_swipe_data)

youve used 239 meal swipes


,Hamp,Berk,Woo,Frank,Roots Cafe,AE IT Mobile Reader
Breakfast,3,0,2,20,30,0
Lunch,27,10,41,7,0,1
Dinner,33,4,34,3,0,0
Late_Night,NA,10,14,NA,NA,NA
Total,63,24,91,30,30,1


In [9]:
dining_dollars = rawdata[rawdata['Account_Name'].isin(["UMass Dining Dollars","C Basic Plan","Student Debit Plan"])]
dining_dollars = dining_dollars[~dining_dollars['Activity_Details'].isin(alldininghalls)]
pattern = r'[0-9]'
dining_dollars.loc[:, 'Amount'] = dining_dollars['Amount'].str.replace('$', '')
dining_dollars.loc[:, 'Amount'] = dining_dollars['Amount'].str.replace('-', '')
dining_dollars.loc[:,'Activity_Details'] = dining_dollars['Activity_Details'].apply(lambda x: re.sub(pattern, '', str(x)))
dining_dollars.loc[:, 'Activity_Details'] = dining_dollars['Activity_Details'].str.strip()
dining_dollars = dining_dollars[~dining_dollars['Activity_Details'].str.contains("WASHER|DRYER|GET FUNDS DEPOSITS|PRNT")]
dining_dollars.loc[:, 'Amount'] = pd.to_numeric(dining_dollars['Amount'])
#display(dining_dollars)
proccessed_dining_dolla_data = pd.DataFrame()
mics_columns = dining_dollars['Activity_Details'].unique()
#display(dining_dollars)
for place in mics_columns:
    
    micsdf = dining_dollars[dining_dollars['Activity_Details'].isin([place])]
    num_breakfast = len(micsdf[micsdf['Date_Time'].dt.time < pd.to_datetime('11:00').time()])
    num_lunch = len(micsdf[(micsdf['Date_Time'].dt.time >= pd.to_datetime('11:00').time()) & (micsdf['Date_Time'].dt.time < pd.to_datetime('16:30').time())])
    num_dinner = len(micsdf[micsdf['Date_Time'].dt.time >= pd.to_datetime('16:30').time()])
    allsum = micsdf['Amount'].sum()
    if(micsdf['Account_Name'] == "C Basic Plan").any():
        allsum *=12
    proccessed_dining_dolla_data[place] = [num_breakfast,num_lunch,num_dinner,len(micsdf),f"${format(allsum,'.2f')}"]
proccessed_dining_dolla_data.rename(index={0: "Breakfast", 1: "Lunch", 2: "Dinner",3:"Total",4:"Total $ Spent"},inplace=True)
display(proccessed_dining_dolla_data)


,Courtside Cafe,Roots Cafe
Breakfast,0,0
Lunch,0,1
Dinner,1,0
Total,1,1
Total $ Spent,$4.00,$9.50


In [10]:
print(f"You have gone to hamp {proccessed_meal_swipe_data['Hamp']['Total']} times")
print(f"You have gone to berk {proccessed_meal_swipe_data['Berk']['Total']} times")
print(f"You have gone to frank {proccessed_meal_swipe_data['Frank']['Total']} times")
print(f"You have gone to woo {proccessed_meal_swipe_data['Woo']['Total']} times")
print(f"youve used {len(mealswipes)} meal swipes")

You have gone to hamp 63 times
You have gone to berk 24 times
You have gone to frank 30 times
You have gone to woo 91 times
youve used 239 meal swipes
